In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data"

In [3]:
# Define column names based on dataset description
names = ['letter', 'x-box', 'y-box', 'width', 'height', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx']


In [4]:
data = pd.read_csv(url,names=names)

In [5]:
#seaparte features and target
X = data.drop('letter',axis=1)
y= data['letter']

In [6]:
#Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y_encoded,test_size=0.2,random_state=42)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [9]:
#DEfine model
model = Sequential([
    # First hidden layer with 64 neurons and ReLU activation function
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    # Second hidden layer with 64 neurons and ReLU activation function
    Dense(64,activation='relu'),
    # Output layer with neurons equal to the number of classes (26 for letters
    # A-Z) and softmax activation function
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [10]:
#Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
#Train the model
model.fit(X_train,y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
450/450 [==============================] - 3s 4ms/step - loss: 2.1815 - accuracy: 0.4149 - val_loss: 1.2696 - val_accuracy: 0.6456
Epoch 2/20
450/450 [==============================] - 1s 2ms/step - loss: 1.0784 - accuracy: 0.6984 - val_loss: 0.9645 - val_accuracy: 0.7481
Epoch 3/20
450/450 [==============================] - 1s 2ms/step - loss: 0.8813 - accuracy: 0.7514 - val_loss: 0.8368 - val_accuracy: 0.7756
Epoch 4/20
450/450 [==============================] - 1s 2ms/step - loss: 0.7697 - accuracy: 0.7830 - val_loss: 0.7463 - val_accuracy: 0.7850
Epoch 5/20
450/450 [==============================] - 1s 2ms/step - loss: 0.6892 - accuracy: 0.7978 - val_loss: 0.6770 - val_accuracy: 0.8125
Epoch 6/20
450/450 [==============================] - 1s 2ms/step - loss: 0.6316 - accuracy: 0.8175 - val_loss: 0.6224 - val_accuracy: 0.8363
Epoch 7/20
450/450 [==============================] - 1s 2ms/step - loss: 0.5755 - accuracy: 0.8312 - val_loss: 0.5645 - val_accuracy: 0.8469
Epoch 

In [17]:
#save the trained model
model.save("letter_recognition_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
#Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy: ',test_acc)

125/125 [==============================] - 0s 1ms/step - loss: 0.2945 - accuracy: 0.9075
Test Accuracy:  0.9075000286102295


In [18]:
#save the label encoder
np.save("label_encoder_classes.npy",label_encoder.classes_)

In [20]:
from tensorflow.keras.models import load_model

In [21]:
# Function to preprocess input dimensions
def preprocess_input(dimensions):
  # Convert dimensions to a numpy array and reshape it to match the input shape
  # of the model
  input_data=np.array(dimensions).reshape(1,-1)
  return input_data

In [22]:
# Function to predict the output using the trained model
def predict_output(model,input_data,label_encoder):
   # Use the model to predict the output
  predicted_class=np.argmax(model.predict(input_data),axis=-1)
  # Convert the predicted class index back to the original letter
  predicted_letter = label_encoder.inverse_transform(predicted_class)
  return predicted_letter[0]

In [25]:
# Load the trained model and label encoder
model=load_model("/content/letter_recognition_model.h5")
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("/content/label_encoder_classes.npy",allow_pickle=True)

In [30]:
# Input dimensions dynamically from the user
dimension_str = input("Enter the dimensions separated by commas: ")
# Convert the input string to a list of integers
dimensions = list(map(int, dimension_str.split(',')))

Enter the dimensions separated by commas: 4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7


In [31]:
# Preprocess the input dimensions
input_data = preprocess_input(dimensions)

In [32]:
# Predict the output
predicted_letter = predict_output(model, input_data,label_encoder)
print("Predicted Letter: ",predicted_letter)

1/1 [==============================] - 0s 25ms/step
Predicted Letter:  S
